In [23]:
# Default Code
!pip3 install torch torchvision

# Code from Colab
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch
torch.__version__

'0.3.0.post4'

In [48]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmp0uly4muo/pubring.gpg' created
gpg: /tmp/tmp0uly4muo/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


KeyboardInterrupt: ignored

In [27]:
!ls
!mkdir -p drive
!ls
!google-drive-ocamlfuse drive
!ls drive

datalab  drive
datalab  drive
fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
사진     2학년	4학년   7alpha.jpg	label_image.py
창고     2alpha.jpg	4alpha.jpg  8alpha.jpg	predict.py
1학년   3학년	5alpha.jpg  9alpha.jpg	retrain.py
1alpha.jpg  3alpha.jpg	6alpha.jpg  eeg.ipynb	workspace


In [51]:
!python3 drive/retrain.py --bottleneck_dir = ./drive/workspace/bottlenecks --model_dir=./drive/workspace/inception --output_graph=./drive/eeg_graph.pb --output_labels=./drive/workspace/eeg_labels.txt --image_dir ./drive/workspace/dataset --how_many_training_steps 10000

2018-07-14 02:39:16.842722: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:897] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-07-14 02:39:16.843235: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1392] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-07-14 02:39:16.843277: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1471] Adding visible gpu devices: 0
2018-07-14 02:39:17.238298: I tensorflow/core/common_runtime/gpu/gpu_device.cc:952] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-07-14 02:39:17.238364: I tensorflow/core/common_runtime/gpu/gpu_device.cc:958]      0 
2018-07-14 02:39:17.238395: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] 0:   N 
2018-07-14 02:39:17.238664: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1084] C

3000 bottleneck files created.
3100 bottleneck files created.
3200 bottleneck files created.
3300 bottleneck files created.
3400 bottleneck files created.
3500 bottleneck files created.
3600 bottleneck files created.
3700 bottleneck files created.
3800 bottleneck files created.
3900 bottleneck files created.
4000 bottleneck files created.
4100 bottleneck files created.
4200 bottleneck files created.
2018-07-14 02:39:44.342580: Step 0: Train accuracy = 33.0%
2018-07-14 02:39:44.342701: Step 0: Cross entropy = 1.901471
2018-07-14 02:39:45.512997: Step 0: Validation accuracy = 32.0% (N=100)
2018-07-14 02:39:46.482755: Step 10: Train accuracy = 28.0%
2018-07-14 02:39:46.482857: Step 10: Cross entropy = 1.724104
2018-07-14 02:39:46.588904: Step 10: Validation accuracy = 17.0% (N=100)
2018-07-14 02:39:47.625356: Step 20: Train accuracy = 81.0%
2018-07-14 02:39:47.625498: Step 20: Cross entropy = 1.679558
2018-07-14 02:39:47.726975: Step 20: Validation accuracy = 76.0% (N=100)
2018-07-14 02:3

2018-07-14 02:40:06.111620: Step 200: Train accuracy = 100.0%
2018-07-14 02:40:06.111721: Step 200: Cross entropy = 0.681962
2018-07-14 02:40:06.205409: Step 200: Validation accuracy = 100.0% (N=100)
2018-07-14 02:40:07.122694: Step 210: Train accuracy = 100.0%
2018-07-14 02:40:07.122800: Step 210: Cross entropy = 0.620676
2018-07-14 02:40:07.218605: Step 210: Validation accuracy = 97.0% (N=100)
2018-07-14 02:40:08.153893: Step 220: Train accuracy = 98.0%
2018-07-14 02:40:08.153984: Step 220: Cross entropy = 0.622778
2018-07-14 02:40:08.242698: Step 220: Validation accuracy = 95.0% (N=100)
2018-07-14 02:40:09.148717: Step 230: Train accuracy = 96.0%
2018-07-14 02:40:09.148820: Step 230: Cross entropy = 0.610085
2018-07-14 02:40:09.243607: Step 230: Validation accuracy = 100.0% (N=100)
2018-07-14 02:40:10.151438: Step 240: Train accuracy = 98.0%
2018-07-14 02:40:10.151568: Step 240: Cross entropy = 0.573855
2018-07-14 02:40:10.242509: Step 240: Validation accuracy = 100.0% (N=100)
2018-

2018-07-14 02:40:28.137206: Step 420: Train accuracy = 100.0%
2018-07-14 02:40:28.137321: Step 420: Cross entropy = 0.395378
2018-07-14 02:40:28.239137: Step 420: Validation accuracy = 98.0% (N=100)
2018-07-14 02:40:29.133671: Step 430: Train accuracy = 99.0%
2018-07-14 02:40:29.133799: Step 430: Cross entropy = 0.341609
2018-07-14 02:40:29.223345: Step 430: Validation accuracy = 99.0% (N=100)
2018-07-14 02:40:30.150675: Step 440: Train accuracy = 100.0%
2018-07-14 02:40:30.150820: Step 440: Cross entropy = 0.385572
2018-07-14 02:40:30.251503: Step 440: Validation accuracy = 100.0% (N=100)
2018-07-14 02:40:31.171927: Step 450: Train accuracy = 97.0%
2018-07-14 02:40:31.172019: Step 450: Cross entropy = 0.353027
2018-07-14 02:40:31.271635: Step 450: Validation accuracy = 100.0% (N=100)
2018-07-14 02:40:32.187234: Step 460: Train accuracy = 98.0%
2018-07-14 02:40:32.187341: Step 460: Cross entropy = 0.358884
2018-07-14 02:40:32.280926: Step 460: Validation accuracy = 100.0% (N=100)
2018-

2018-07-14 02:40:50.292959: Step 640: Train accuracy = 100.0%
2018-07-14 02:40:50.293062: Step 640: Cross entropy = 0.293265
2018-07-14 02:40:50.382120: Step 640: Validation accuracy = 99.0% (N=100)
2018-07-14 02:40:51.262920: Step 650: Train accuracy = 100.0%
2018-07-14 02:40:51.263032: Step 650: Cross entropy = 0.257733
2018-07-14 02:40:51.355660: Step 650: Validation accuracy = 100.0% (N=100)
2018-07-14 02:40:52.255253: Step 660: Train accuracy = 100.0%
2018-07-14 02:40:52.255342: Step 660: Cross entropy = 0.230742
2018-07-14 02:40:52.348045: Step 660: Validation accuracy = 100.0% (N=100)
2018-07-14 02:40:53.248889: Step 670: Train accuracy = 99.0%
2018-07-14 02:40:53.248986: Step 670: Cross entropy = 0.247477
2018-07-14 02:40:53.339860: Step 670: Validation accuracy = 100.0% (N=100)
2018-07-14 02:40:54.258633: Step 680: Train accuracy = 100.0%
2018-07-14 02:40:54.258753: Step 680: Cross entropy = 0.253475
2018-07-14 02:40:54.355539: Step 680: Validation accuracy = 99.0% (N=100)
201

...

2018-07-14 02:54:59.624341: Step 8980: Validation accuracy = 100.0% (N=100)
2018-07-14 02:55:00.572090: Step 8990: Train accuracy = 100.0%
2018-07-14 02:55:00.572178: Step 8990: Cross entropy = 0.030355
2018-07-14 02:55:00.668317: Step 8990: Validation accuracy = 100.0% (N=100)
2018-07-14 02:55:01.590457: Step 9000: Train accuracy = 100.0%
2018-07-14 02:55:01.590546: Step 9000: Cross entropy = 0.022724
2018-07-14 02:55:01.686405: Step 9000: Validation accuracy = 100.0% (N=100)
2018-07-14 02:55:02.599660: Step 9010: Train accuracy = 100.0%
2018-07-14 02:55:02.599764: Step 9010: Cross entropy = 0.030073
2018-07-14 02:55:02.695757: Step 9010: Validation accuracy = 100.0% (N=100)
2018-07-14 02:55:03.599092: Step 9020: Train accuracy = 100.0%
2018-07-14 02:55:03.599181: Step 9020: Cross entropy = 0.028072
2018-07-14 02:55:03.692504: Step 9020: Validation accuracy = 100.0% (N=100)
2018-07-14 02:55:04.697107: Step 9030: Train accuracy = 100.0%
2018-07-14 02:55:04.697245: Step 9030: Cross entr

2018-07-14 02:55:22.147798: Step 9200: Validation accuracy = 100.0% (N=100)
2018-07-14 02:55:23.071366: Step 9210: Train accuracy = 99.0%
2018-07-14 02:55:23.071540: Step 9210: Cross entropy = 0.046240
2018-07-14 02:55:23.175880: Step 9210: Validation accuracy = 100.0% (N=100)
2018-07-14 02:55:24.106950: Step 9220: Train accuracy = 100.0%
2018-07-14 02:55:24.107057: Step 9220: Cross entropy = 0.031196
2018-07-14 02:55:24.203074: Step 9220: Validation accuracy = 100.0% (N=100)
2018-07-14 02:55:25.126598: Step 9230: Train accuracy = 100.0%
2018-07-14 02:55:25.126705: Step 9230: Cross entropy = 0.026214
2018-07-14 02:55:25.223146: Step 9230: Validation accuracy = 100.0% (N=100)
2018-07-14 02:55:26.145499: Step 9240: Train accuracy = 99.0%
2018-07-14 02:55:26.145586: Step 9240: Cross entropy = 0.040264
2018-07-14 02:55:26.236523: Step 9240: Validation accuracy = 100.0% (N=100)
2018-07-14 02:55:27.128591: Step 9250: Train accuracy = 100.0%
2018-07-14 02:55:27.128699: Step 9250: Cross entrop

2018-07-14 02:55:44.421042: Step 9420: Validation accuracy = 100.0% (N=100)
2018-07-14 02:55:45.355717: Step 9430: Train accuracy = 100.0%
2018-07-14 02:55:45.355806: Step 9430: Cross entropy = 0.018898
2018-07-14 02:55:45.453517: Step 9430: Validation accuracy = 100.0% (N=100)
2018-07-14 02:55:46.472391: Step 9440: Train accuracy = 100.0%
2018-07-14 02:55:46.472486: Step 9440: Cross entropy = 0.028640
2018-07-14 02:55:46.582362: Step 9440: Validation accuracy = 100.0% (N=100)
2018-07-14 02:55:47.626724: Step 9450: Train accuracy = 100.0%
2018-07-14 02:55:47.626913: Step 9450: Cross entropy = 0.042960
2018-07-14 02:55:47.741745: Step 9450: Validation accuracy = 100.0% (N=100)
2018-07-14 02:55:48.768971: Step 9460: Train accuracy = 100.0%
2018-07-14 02:55:48.769102: Step 9460: Cross entropy = 0.029755
2018-07-14 02:55:48.873151: Step 9460: Validation accuracy = 100.0% (N=100)
2018-07-14 02:55:49.794233: Step 9470: Train accuracy = 100.0%
2018-07-14 02:55:49.794364: Step 9470: Cross entr

2018-07-14 02:56:07.384326: Step 9640: Validation accuracy = 100.0% (N=100)
2018-07-14 02:56:08.319189: Step 9650: Train accuracy = 100.0%
2018-07-14 02:56:08.319291: Step 9650: Cross entropy = 0.021670
2018-07-14 02:56:08.416313: Step 9650: Validation accuracy = 100.0% (N=100)
2018-07-14 02:56:09.339688: Step 9660: Train accuracy = 99.0%
2018-07-14 02:56:09.339793: Step 9660: Cross entropy = 0.037144
2018-07-14 02:56:09.436915: Step 9660: Validation accuracy = 100.0% (N=100)
2018-07-14 02:56:10.375646: Step 9670: Train accuracy = 100.0%
2018-07-14 02:56:10.375754: Step 9670: Cross entropy = 0.027908
2018-07-14 02:56:10.475494: Step 9670: Validation accuracy = 100.0% (N=100)
2018-07-14 02:56:11.412020: Step 9680: Train accuracy = 100.0%
2018-07-14 02:56:11.412122: Step 9680: Cross entropy = 0.017147
2018-07-14 02:56:11.510016: Step 9680: Validation accuracy = 100.0% (N=100)
2018-07-14 02:56:12.453303: Step 9690: Train accuracy = 100.0%
2018-07-14 02:56:12.453415: Step 9690: Cross entro

2018-07-14 02:56:29.680474: Step 9860: Validation accuracy = 100.0% (N=100)
2018-07-14 02:56:30.621354: Step 9870: Train accuracy = 100.0%
2018-07-14 02:56:30.621476: Step 9870: Cross entropy = 0.044572
2018-07-14 02:56:30.720312: Step 9870: Validation accuracy = 100.0% (N=100)
2018-07-14 02:56:31.637184: Step 9880: Train accuracy = 100.0%
2018-07-14 02:56:31.637278: Step 9880: Cross entropy = 0.022216
2018-07-14 02:56:31.727505: Step 9880: Validation accuracy = 100.0% (N=100)
2018-07-14 02:56:32.623888: Step 9890: Train accuracy = 100.0%
2018-07-14 02:56:32.623987: Step 9890: Cross entropy = 0.022724
2018-07-14 02:56:32.716690: Step 9890: Validation accuracy = 100.0% (N=100)
2018-07-14 02:56:33.640346: Step 9900: Train accuracy = 99.0%
2018-07-14 02:56:33.640457: Step 9900: Cross entropy = 0.038075
2018-07-14 02:56:33.730047: Step 9900: Validation accuracy = 100.0% (N=100)
2018-07-14 02:56:34.636260: Step 9910: Train accuracy = 100.0%
2018-07-14 02:56:34.636405: Step 9910: Cross entro

In [54]:
!python3 drive/predict.py drive/1alpha.jpg
!python3 drive/predict.py drive/2alpha.jpg
!python3 drive/predict.py drive/1low_beta.jpg
!python3 drive/predict.py drive/2low_beta.jpg
!python3 drive/predict.py drive/1high_beta.jpg
!python3 drive/predict.py drive/2high_beta.jpg
!python3 drive/predict.py drive/1gamma.jpg
!python3 drive/predict.py drive/2gamma.jpg
!python3 drive/predict.py drive/1smr.jpg
!python3 drive/predict.py drive/2smr.jpg
!python3 drive/predict.py drive/1theta.jpg
!python3 drive/predict.py drive/2theta.jpg
!python3 drive/predict.py drive/1delta.jpg
!python3 drive/predict.py drive/2delta.jpg

=== 예측 결과 ===
alpha (83.45%)
delta (3.08%)
gamma (6.68%)
high beta (0.05%)
low beta (2.25%)
smr (0.08%)
theta (4.41%)
=== 예측 결과 ===
alpha (94.63%)
delta (0.25%)
gamma (0.13%)
high beta (0.01%)
low beta (1.07%)
smr (0.41%)
theta (3.50%)
=== 예측 결과 ===
alpha (0.01%)
delta (0.21%)
gamma (0.03%)
high beta (0.21%)
low beta (98.96%)
smr (0.08%)
theta (0.51%)
=== 예측 결과 ===
alpha (0.05%)
delta (0.10%)
gamma (0.01%)
high beta (0.01%)
low beta (99.81%)
smr (0.01%)
theta (0.02%)
=== 예측 결과 ===
alpha (0.04%)
delta (0.26%)
gamma (0.70%)
high beta (98.91%)
low beta (0.02%)
smr (0.07%)
theta (0.01%)
=== 예측 결과 ===
alpha (0.04%)
delta (0.81%)
gamma (1.69%)
high beta (97.23%)
low beta (0.02%)
smr (0.20%)
theta (0.01%)
=== 예측 결과 ===
alpha (0.09%)
delta (0.21%)
gamma (97.28%)
high beta (0.25%)
low beta (2.05%)
smr (0.01%)
theta (0.11%)
=== 예측 결과 ===
alpha (0.07%)
delta (0.04%)
gamma (98.81%)
high beta (0.04%)
low beta (0.36%)
smr (0.01%)
theta (0.68%)
=== 예측 결과 ===
alpha (2.92%)
delta (0.67%)
gamma (1.87%)


In [44]:
!ls drive

사진     1high_beta.jpg  2delta.jpg	    3학년	    retrain.py
창고     1low_beta.jpg   2gamma.jpg	    4학년	    workspace
1학년   1smr.jpg	    2high_beta.jpg  eeg_graph.pb
1alpha.jpg  1theta.jpg	    2low_beta.jpg   eeg.ipynb
1delta.jpg  2학년	    2smr.jpg	    label_image.py
1gamma.jpg  2alpha.jpg	    2theta.jpg	    predict.py
